# FEATURE MAP VISUALIZATION

## Import Packages

In [ ]:
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from keras import models
import keras

import os
os.environ['CUDA_VISIBLE_DEVICES']='99'

In [ ]:
from keras.applications.nasnet import NASNetLarge
from keras.applications.resnet50 import ResNet50

# model_nas = NASNetLarge(weights='imagenet')
# model_res = ResNet50(weights='imagenet')

In [ ]:
# 이미지 로드
img_path = './sample_imgs/cat_dog.png'

img = image.load_img(img_path, target_size=(331,331))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor,axis=0)
img_tensor /= 255.

In [ ]:
# 모델 로드
model_path = './logs/diebonder/logs_200210/NASNetLarge/train_best.h5'
model = models.load_model(model_path)

## Feature map 시각화

In [ ]:
activaction_layers = []
for l in layers:
    if isinstance(l,keras.layers.Activation):
        activaction_layers.append(l)
        
layer_outputs = [model.layers[0].layers[-2].output]
activation_model = models.Model(inputs=model.layers[0].get_input_at(0), outputs= layer_outputs)

activations = activation_model.predict(img_tensor)
print (len(activactions))

In [ ]:
activaction_layers = []
layer_names = []
layer_outputs = []

for l in layers:
    if isinstance(l,keras.layers.Activation):
        activaction_layers.append(l)
        layer_outputs.append(l.output)
        layer_names.append(l.name)
        
activation_model = models.Model(inputs=model.layers[0].get_input_at(0), outputs= layer_outputs)
activactions = activation_model.predict(img_tensor)

In [ ]:
# feature map 출력
images_per_row = 16

for layer_name, activaction_layer in zip(layer_names, activactions):
    n_features = activaction_layer.shape[-1]
    size = activaction_layer.shape[1]
    
    n_cols = n_features // images_per_row
    dp_grid = np.zeros((size*n_cols, images_per_row*size))
    
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = activaction_layer[0,:,:,col*images_per_row+row]
            
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image,0,255).astype('uint8')
            
            dp_grid[col*size : (col+1)*size,
                    row*size : (row+1)*size] = channel_image
            
    scale = 1./size
    plt.figure(figsize = (scale*dp_grid.shape[1],
                          scale*dp_grid.shape[0]))
            
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(dp_grid,aspect='auto', cmap='viridis')
plt.show()